## "A Bossy Sort of Voice"
#### A study of sexism in the Harry Potter series using Natural Language Processing

My eldest son is almost 6 and loves the Harry Potter series of books.  But, as I read them for the first time with him as a bedtime story, I noticed something hadn't expected.  

The sexism.

The sexism critique of the Harry Potter novels is not a new one - many people have (written)[https://www.bustle.com/articles/136244-the-5-least-feminist-moments-in-harry-potter] excellent articles about Ron's treatment of Hermoine, the portrayl of other female characters as cold or incompetent or promiscuous.  But there were two types of analyses I didn't see: a quantitative or linguistics based one, or something that looked at how the author herself portrays female characters in a biased light in how they speak

My hope in doing this is that people will use some of these tools to look more critically at the language of literature we love and the popular press for signs of gender, racial, and other bias.

#### Three hypotheses

In this project, I will test three hypotheses:

1. Female characters are referred to by the narrator with sexist words throughout the series by the narrator, while males are not described with that same language.

2. The narrator will use more sexist words when describing the female characters than other characters will use when talking about them.

3. Sexism as defined above will decline in both dialog and narration as the series progresses.


Taking this approach requires tools to do the following:

1. Process text from all 7 'Harry Potter' books.

2. Distinguish parts of speech, like nouns, verbs, and adjectives.

3. Seperate dialog from narration.

4. Be able to tell female characters from male characters

4. 

4. Reduce sexist words to their root forms (e.g. 'shriller' should be equivalent to 'shrill').


#### Sexist language
Since sexism in language is sometimes very subtle, a challenge for this project is identifying what words or combinations of words and their context should be considered biased.

After some searching, I found a couple of excellent lists.  Because the Harry Potter series is written in English by a British writer, I focused on sources from the UK and countries in the Commonwealth.  Using this blog post by a [New Zealand blogger](http://sacraparental.com/2016/05/14/everyday-misogyny-122-subtly-sexist-words-women/) I had a first set of words and some excellent categories to begin with.  I found a number of [other](http://time.com/4268325/history-calling-women-shrill/) excellent articles about sexism in language, which I used to add to the `sexist_words` Python dictionary below.

In [2]:
sexist_words ={ 
    'assertiveness': ['bossy', 'abrasive', 'ball-buster', 'aggressive', 'shrill', 'bolshy', 'intense', 'stroppy', 'forward', 'mannish', 'strident', 'know-it-all'],
    'behavior' : ['cackle', 'shriek', 'caterwaul', 'yowl', 'screech','gossip', 'dramatic', 'catty', 'bitch', 'nag', 'cold', 'icy', 'shrew', 'humorless', 'man-hater', 'banshee', 'fishwife', 'lippy', 'ditzy', 'diva', 'prima donna', 'feisty', 'ladylike', 'bubbly', 'vivaious', 'flirt', 'sass', 'chatty', 'demure', 'modest', 'emotional', 'hysterical', 'hormonal', 'menstrual', 'flaky', 'moody', 'over-sensitive'],
    'sexuality': ['slut', 'trollop', 'frigid', 'easy', 'tease', 'loose', 'man-eater', 'prude', 'curvy', 'cheap', 'frumpy', 'faded', 'mousey', 'clotheshorse', 'cow', 'hag'],
    'relationship': ['spinster', 'barren', 'housewife', 'houseproud', 'soccer mom', 'mistress', 'kept woman'],
    'praise': ['caring', 'compassionate', 'hard-working', 'conscientious', 'dependable', 'diligent', 'dedicated', 'tactful', 'interpersonal', 'warm', 'helpful'],
}

#### Step 3: Parsing the text
